In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer

In [ ]:
emotions_data = pd.read_csv("/content/sample_data/Go_Emotions.csv")
emotions_data.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,yeah dude I love my 9(???) with the welded bum...,efavqq3,snailgazer,thatHappened,t3_aknw1o,t3_aknw1o,1548800100,62,False,0,...,1,0,0,0,0,0,0,0,0,0
1,It goes away and can be treated. Once treated ...,edc3wxr,paperconservation101,bestoflegaladvice,t3_acu5er,t1_edb3sxb,1546729300,30,False,0,...,0,0,0,0,0,0,0,0,0,0
2,Is there a quiz version of this where I can an...,ef7mc9y,pyrohmstr,chicago,t3_aksvix,t3_aksvix,1548715300,4,False,0,...,0,0,0,0,0,0,0,0,0,0
3,Nyaw :) imagine all the nice things he’ll have...,eerla6a,ino_y,DeadBedrooms,t3_aiwx13,t1_eerjaxz,1548252000,5,False,0,...,0,0,0,0,0,0,0,0,0,0
4,"don't worry, the funding will arrive the year ...",eep877a,Real_PoopyButthole,worldnews,t3_ailr22,t3_ailr22,1548174500,60,False,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
emotions_data.dtypes

,0
text,object
id,object
author,object
subreddit,object
link_id,object
parent_id,object
created_utc,int64
rater_id,int64
example_very_unclear,bool
admiration,int64


In [ ]:
emotions_data.example_very_unclear = emotions_data.example_very_unclear.replace({True: 1, False: 0})

<ipython-input-5-6bbd94a230db>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  emotions_data.example_very_unclear = emotions_data.example_very_unclear.replace({True: 1, False: 0})


In [ ]:
emotions_data = emotions_data.drop(columns=['id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id'])

In [ ]:
emotions_data.dtypes

,0
text,object
example_very_unclear,int64
admiration,int64
amusement,int64
anger,int64
annoyance,int64
approval,int64
caring,int64
confusion,int64
curiosity,int64


In [ ]:
emotion_labels = emotions_data.columns[1:].tolist()
emotion_labels

['example_very_unclear',
 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [ ]:
emotions_dataset = Dataset.from_pandas(emotions_data)
emotions_dataset

Dataset({
    features: ['text', 'example_very_unclear', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
    num_rows: 211225
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

def process_labels(emotions):
    labels = [float(emotions[label]) for label in emotion_labels]
    emotions["labels"] = torch.tensor(labels, dtype = torch.float32)
    return emotions

In [ ]:
emotions_dataset = emotions_dataset.map(tokenize_function, batched = True)
emotions_dataset.shape

Map:   0%|          | 0/211225 [00:00<?, ? examples/s]

(211225, 33)

In [ ]:
emotions_dataset = emotions_dataset.map(process_labels)
emotions_dataset.shape

Map:   0%|          | 0/211225 [00:00<?, ? examples/s]

(211225, 34)

In [ ]:
train_test_split = emotions_dataset.train_test_split(test_size=0.2, seed=42)
emotions_train_dataset = train_test_split["train"]
emotions_test_dataset = train_test_split["test"]

In [ ]:
emotions_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
emotions_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
from transformers import AutoModelForSequenceClassification

emotion_classifier_model = AutoModelForSequenceClassification.from_pretrained(
    "nateraw/bert-base-uncased-emotion",
    num_labels=len(emotion_labels),
    problem_type = "multi_label_classification",
    ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nateraw/bert-base-uncased-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([29, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([29]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda")
device

device(type='cuda')

In [ ]:
from transformers import Trainer, TrainingArguments

#emotion_classifier_model.to(device)

training_args = TrainingArguments(

    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=50
)


trainer = Trainer(
    model=emotion_classifier_model,
    args=training_args,
    train_dataset=emotions_train_dataset,
    eval_dataset=emotions_test_dataset
)

#trainer.train()


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

import numpy as np
from sklearn.metrics import accuracy_score

predictions = trainer.predict(emotions_test_dataset)
predicted_labels = (predictions.predictions > 0.5).astype(int)
true_labels = np.array(emotions_test_dataset['labels'])

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy on test set: {accuracy}")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gprashanth7301 (gprashanth7301-university-at-albany) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Accuracy on test set: 0.13809918333530596


In [ ]:
path = "emotion_classifier_v2_dict_model"
emotion_classifier_model.save_pretrained(path)

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the saved model and tokenizer
model_path = "emotion_classifier_v2_dict_model"
tokenizer = AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion")
emotion_classifier_model = AutoModelForSequenceClassification.from_pretrained(model_path)

print("Model and tokenizer loaded successfully!")

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Model and tokenizer loaded successfully!


In [ ]:
device = torch.device("cuda")
device

device(type='cuda')

In [ ]:
from transformers import pipeline

emotion_classifier_model.to(device)

emotion_classifier = pipeline("text-classification", model=emotion_classifier_model, tokenizer=tokenizer)


Device set to use cuda:0


In [ ]:
def convert_label_to_emotion(predicted_emotion):

    label_to_emotion = {f"LABEL_{i}": emotion for i, emotion in enumerate(emotion_labels)}

    emotion_scores = {}
    for item in predicted_emotion:
        label = item['label']
        score = item['score']

        if label in label_to_emotion:
            emotion = label_to_emotion[label]
            emotion_scores[emotion] = score
        else:
            print(f"Warning: Label '{label}' not found in the label mapping.")

    return emotion_scores

In [ ]:

def classify_emotion(emotion_scores):

    positive_emotions = ["admiration", "amusement", "approval", "caring", "curiosity", "desire", "excitement", "gratitude", "joy", "love", "optimism", "pride", "relief"]
    negative_emotions = ["anger", "annoyance", "disappointment", "disapproval", "disgust", "embarrassment", "fear", "grief", "nervousness", "remorse", "sadness"]
    mixed_emotions = ["example_very_unclear", "confusion", "neutral", "surprise", "realization"]

    positive_score = sum(emotion_scores.get(emotion, 0) for emotion in positive_emotions)
    negative_score = sum(emotion_scores.get(emotion, 0) for emotion in negative_emotions)
    neutral_score = sum(emotion_scores.get(emotion, 0) for emotion in mixed_emotions)

    if positive_score > negative_score:
        return "positive"
    elif negative_score > positive_score:
        return "negative"
    else:
        return "mixed"

print(convert_label_to_emotion(emotion_classifier("I dont know how i feel")))
scores = convert_label_to_emotion(emotion_classifier("I dont know how i feel"))
print(classify_emotion(scores))

print(convert_label_to_emotion(emotion_classifier("I want to kill myself")))
scores = convert_label_to_emotion(emotion_classifier("I want to kill myself"))
print(classify_emotion(scores))

print(convert_label_to_emotion(emotion_classifier("My life takes a lot turns, somethings fade away when they come near to me")))
scores = convert_label_to_emotion(emotion_classifier("My life takes a lot turns, somethings fade away when they come near to me"))
print(classify_emotion(scores))

print(convert_label_to_emotion(emotion_classifier("I like my girlfriend")))
scores = convert_label_to_emotion(emotion_classifier("I like my girlfriend"))
print(classify_emotion(scores))

print(convert_label_to_emotion(emotion_classifier("I am very excited to attend funeral tomorrow")))
scores = convert_label_to_emotion(emotion_classifier("I am very excited to attend funeral tomorrow"))
print(classify_emotion(scores))


{'confusion': 0.6632118225097656}
mixed
{'anger': 0.22856493294239044}
negative
{'neutral': 0.3494170904159546}
mixed
{'love': 0.6754611134529114}
positive
{'excitement': 0.7138499617576599}
positive
